In [1]:
import numpy as np
import cv2

In [2]:
low_threshold = 0
iteration_num = 5
src = cv2.imread('img.png')

def update():
	img = src.copy()
	gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	_, thresh = cv2.threshold(gray_img, low_threshold, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
	kernel = np.ones((3, 3), np.uint8)
	sure_bg = cv2.dilate(thresh, kernel, iterations=iteration_num)
	dist_transform = cv2.distanceTransform(sure_bg, 1, 5)
	dist_transform = np.uint8(dist_transform)
	_, sure_fg = cv2.threshold(dist_transform, low_threshold, 255, 0)
	sure_fg = np.uint8(sure_fg)
	unknown = cv2.subtract(sure_bg, sure_fg)
	_, markers = cv2.connectedComponents(sure_fg)
	markers = markers + 1
	markers[unknown == 255] = 0
	markers = cv2.watershed(img, markers)
	img[markers == -1] = (0, 0, 255)
	cv2.imshow("img", img)

def change_low_threshold(val):
	global low_threshold
	low_threshold = val
	update()


def change_iteration_num(val):
	global iteration_num
	iteration_num = val
	update()

cv2.namedWindow('Settings')
cv2.createTrackbar('threshold', 'Settings', 20, 255, change_low_threshold)
cv2.createTrackbar('iterations', 'Settings', 5, 100, change_iteration_num)
cv2.waitKey(0)
cv2.destroyAllWindows()